In [ ]:
%flink.ssql
SELECT *
FROM stock_table
MATCH_RECOGNIZE(
	PARTITION BY ticker
	ORDER BY event_time
	MEASURES
	    A.event_time AS event_time, 
		A.`date` AS initialPriceDate, 
		C. `date` AS dropDate, 
		(100 * (A.close_price - C.close_price) / A.close_price) AS dropPercentage, 
		A.close_price AS initialPrice, 
		C.close_price AS lastPrice
	ONE ROW PER MATCH
	AFTER MATCH SKIP PAST LAST ROW
	PATTERN(A B{0, 2} C)
	DEFINE
		B AS -100 * (B.close_price - A.close_price) / A.close_price < 8.0,
	    C AS 100 * (A.close_price - C.close_price) / A.close_price >= 8.0
)